In [ ]:
!nvidia-smi

In [ ]:
# Installation Library

!pip install transformers datasets sentencepiece rouge 

In [ ]:
import torch
import pandas as pd
from rouge import Rouge
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

In [ ]:
torch.manual_seed(0)

In [ ]:
dataset = load_dataset("billsum")

Using custom data configuration default


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /root/.cache/huggingface/datasets/billsum/default/3.0.0/d1e95173aed3acb71327864be74ead49b578522e4c7206048b2f2e5351b57959. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
test_cases =  dataset['ca_test']

In [ ]:
case1 = test_cases[1]['text']
sum1 =  test_cases[1]['summary']

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


checkpoint = "google/bigbird-pegasus-large-bigpatent"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = BigBirdPegasusForConditionalGeneration.from_pretrained(checkpoint).to(device)

Downloading:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

In [ ]:
input_ids = tokenizer(case1, return_tensors='pt').to("cuda")

In [ ]:
input_ids

{'input_ids': tensor([[139, 200, 113,  ..., 445, 107,   1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}

In [ ]:
sequences = model.generate(**input_ids)

summary = tokenizer.batch_decode(sequences,
                                 skip_special_tokens=True)

/usr/local/lib/python3.7/dist-packages/transformers/models/bigbird_pegasus/modeling_bigbird_pegasus.py:805: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  * num_indices_to_pick_from


In [ ]:
standard_summary = sum1
system_summary = summary[0]

In [ ]:
rouge = Rouge()
score = rouge.get_scores(system_summary, system_summary.split('.')[0])
pd.DataFrame(score[0]).set_index([['recall','precision','f-measure']])

,rouge-1,rouge-2,rouge-l
recall,1.000000,1.000000,1.000000
precision,0.785714,0.714286,0.785714
f-measure,0.880000,0.833333,0.880000


Test DataSet

In [ ]:
CasesText = test_cases['text']
GoldSummary = test_cases['summary']

In [ ]:
len(CasesText), len(GoldSummary)

(1237, 1237)

In [ ]:
# CasesText[:500]
# CasesText[500:800]
# CasesText[800:]

In [ ]:
SystemSummary = []

for i, case in enumerate(CasesText[800:]):
    
    input_ids = tokenizer(case, return_tensors='pt').to("cuda")
    sequences = model.generate(**input_ids)

    summary = tokenizer.batch_decode(sequences,
                                 skip_special_tokens=True)
    SystemSummary.append(summary)
    print(i)

In [ ]:
SystemSummaryFinal = []

for i in SystemSummary:
  SystemSummaryFinal.append((i[0]))

In [ ]:
len(SystemSummaryFinal)

437

In [ ]:
Summaries = pd.DataFrame(list(zip(GoldSummary[800:], SystemSummaryFinal)), columns =['GoldSummary', 'SystemSummary'])

In [ ]:
path = "/content/drive/MyDrive/LegSuM/Data/Data by Models/"

Summaries.to_csv(path + "bigbird3.csv", header=True, index=False)

**BART Model Score on Test**

In [ ]:
path = "/content/drive/MyDrive/LegSuM/Data/Data by Models/"

In [ ]:
file1 = pd.read_csv(path + "bigbird1.csv")
file2 = pd.read_csv(path + "bigbird2.csv")
file3 = pd.read_csv(path + "bigbird3.csv")

In [ ]:
file1.shape, file2.shape, file3.shape, 

((500, 2), (300, 2), (437, 2))

In [ ]:
Bigbird = pd.concat([file1, file2, file3])

In [ ]:
Bigbird.reset_index(inplace= True, drop=True)

In [ ]:
Bigbird.sample(5)

,GoldSummary,SystemSummary
1168,The California Constitution grants the retirem...,The following people have taken the lead in ad...
1225,"Existing law requires a solicitor or seller, p...",A method and system for collecting information...
944,(1) The existing California Civil Liberties Pu...,A test for determining whether an eligible for...
853,Existing law requires the State Registrar to a...,A method for preparing and maintaining compreh...
812,Existing law requires the adopted course of st...,An urgency statute to protect public schools f...


In [ ]:
system_summary = Bigbird['SystemSummary']
standard_summary = Bigbird['GoldSummary']

In [ ]:
rouge = Rouge()
score = rouge.get_scores(system_summary, standard_summary, avg=True)

In [ ]:
BigbirdRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])

In [ ]:
BigbirdRouge

,rouge-1,rouge-2,rouge-l
recall,0.159081,0.062249,0.142693
precision,0.534836,0.262118,0.487308
f-measure,0.229950,0.092690,0.207045


In [ ]:
path = "/content/drive/MyDrive/LegSuM/scores/"

BigbirdRouge.to_csv(path + "BigbirdRouge.csv", header=True)